In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import cv2
import tqdm
import numpy as np
import pandas as pd
import preprocess_crop
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
TRAIN_IMAGES = 'data/training/images/'
TRAIN_LABELS = 'data/training/labels.csv'
TEST_IMAGES = 'data/images/'
SAMPLE = 'data/sample_submission.csv'

In [ ]:
df_train_labels = pd.read_csv(TRAIN_LABELS)
df_train_labels.head()

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(16):
    filename, xRot = df_train_labels.iloc[i]
    filepath = os.path.join(TRAIN_IMAGES, filename)
    image = cv2.imread(filepath)
    image = cv2.resize(image, (224, 224))
    plt.subplot(4, 4, i+1)
    plt.axis('off')
    plt.title("xRot: %.3f" % (xRot))
    plt.imshow(image)

In [ ]:
datagen = ImageDataGenerator(shear_range=0.2,
                             zoom_range=0.2,
                             preprocessing_function=preprocess_input,
                             validation_split=0.2)

In [ ]:
SIZE = (224, 224, 3)

In [ ]:
train_generator = datagen.flow_from_dataframe(dataframe=df_train_labels,
                                              directory='data/training/images/',
                                              x_col='filename',
                                              y_col='xRot',
                                              subset='training',
                                              batch_size=8,
                                              seed=42,
                                              shuffle=True,
                                              class_mode='raw',
                                              target_size=SIZE[:2])

val_generator = datagen.flow_from_dataframe(dataframe=df_train_labels,
                                              directory='data/training/images/',
                                              x_col='filename',
                                              y_col='xRot',
                                              subset='validation',
                                              batch_size=8,
                                              seed=42,
                                              shuffle=True,
                                              class_mode='raw',
                                              target_size=SIZE[:2])

In [ ]:
resnet_base = ResNet50(include_top=False, input_shape=SIZE, pooling='max')
for layer in resnet_base.layers:
    layer.trainable = True

In [ ]:
resnet_output = resnet_base.get_layer('conv5_block3_out').output
layer = Flatten()(resnet_output)
layer = Dense(units=256, activation='relu')(layer)
layer = Dense(units=64, activation='relu')(layer)
layer = Dense(units=16, activation='relu')(layer)
layer = Dense(units=4, activation='relu')(layer)
layer = Dense(units=1, activation='linear')(layer)

In [ ]:
model = Model(resnet_base.input, layer)
model.compile(optimizer='RMSprop', loss='mse', metrics=['mse', 'mae'])
model.summary()

In [ ]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VAL = val_generator.n // val_generator.batch_size

In [ ]:
checkpoint = ModelCheckpoint('ckpnt_{epoch:02d}.h5', save_weights_only=True, period=2)

In [ ]:
model.fit_generator(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VAL,
                    epochs=15,
                    callbacks=[checkpoint])

In [ ]:
model.load_weights('ckpnt_14.h5')

In [ ]:
val_files = val_generator.filenames

In [ ]:
y_val = []
y_pred = []
for file in val_files:
    image = cv2.imread(TRAIN_IMAGES + file)
    image = cv2.resize(image, (224, 224))
    image = np.reshape(image, (1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    
    y_pred.append(model.predict(image)[0][0])
    xrot = df_train_labels[df_train_labels['filename'] == file]['xRot'].item()
    y_val.append(xrot)
    
    del image

In [ ]:
y_val = np.array(y_val)
y_pred = np.array(y_pred)

In [ ]:
print(mean_absolute_error(y_val, y_pred))
print(mean_squared_error(y_val, y_pred))

In [ ]:
test_files = os.listdir(TEST_IMAGES)
test_files.sort()

In [ ]:
predictions = []
for file in test_files:
    image = cv2.imread(TEST_IMAGES + file)
    image = cv2.resize(image, (224, 224))
    image = np.reshape(image, (1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    
    pred = model.predict(image)
    predictions.append(pred[0][0])
    
    del image, pred

In [ ]:
submissions = pd.DataFrame(predictions, columns=['xRot'])
submissions['filename'] = test_files

In [ ]:
submissions.head()

In [ ]:
submissions.to_csv('data/submission.csv', index=False)